#Installing Dependencies

In [1]:
!pip install --upgrade onn

Requirement already up-to-date: onn in /usr/local/lib/python3.6/dist-packages (0.1.2)


##Importing Dependencies

In [0]:
from onn.OnlineNeuralNetwork import ONN
from onn.OnlineNeuralNetwork import ONN_THS
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score
import numpy as np

## Initializing Network

In [0]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10)

##Creating Fake Classification Dataset

In [0]:
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=4, n_redundant=0, n_classes=10,
                           n_clusters_per_class=1, class_sep=3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

##Learning and predicting at the same time

In [5]:
for i in range(len(X_train)):
  onn_network.partial_fit(np.asarray([X_train[i, :]]), np.asarray([y_train[i]]))
  
  if i % 1000 == 0:
    predictions = onn_network.predict(X_test)
    print("Online Accuracy: {}".format(accuracy_score(y_test, predictions)))

Online Accuracy: 0.14513333333333334
Alpha:[0.8394298  0.04014252 0.04014252 0.04014252 0.04014252]
Training Loss: 1.3401858
Online Accuracy: 0.9665333333333334
Alpha:[0.83981496 0.04004624 0.04004624 0.04004624 0.04004624]
Training Loss: 0.47074658
Online Accuracy: 0.9748
Alpha:[0.83985054 0.04003733 0.04003733 0.04003733 0.04003733]
Training Loss: 0.3416147
Online Accuracy: 0.9767333333333333
Alpha:[0.8399445  0.04001385 0.04001385 0.04001385 0.04001385]
Training Loss: 0.27886173
Online Accuracy: 0.9772
Alpha:[0.839963   0.04000923 0.04000923 0.04000923 0.04000923]
Training Loss: 0.31251636
Online Accuracy: 0.9772666666666666
Alpha:[0.8397189  0.04007026 0.04007026 0.04007026 0.04007026]
Training Loss: 0.22847703
Online Accuracy: 0.9803333333333333
Alpha:[0.83356273 0.04162288 0.04159797 0.04160155 0.04161485]
Training Loss: 0.26110825
Online Accuracy: 0.9798
Alpha:[0.8396703 0.0400824 0.0400824 0.0400824 0.0400824]
Training Loss: 0.20787847
Online Accuracy: 0.98
Alpha:[0.83994615 0.

# Learning in batch with CUDA

In [6]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10, batch_size=10, use_cuda=True)

Using CUDA :]


In [0]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):

  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __len__(self):
      return len(self.X)

  def __getitem__(self, idx):
      X = self.X[idx],
      Y = self.Y[idx]

      return X, Y
    
transformed_dataset = Dataset(X_train, y_train)
dataloader = DataLoader(transformed_dataset, batch_size=10,shuffle=True, num_workers=1)

In [9]:
for local_X, local_y in dataloader: 
  onn_network.partial_fit(np.squeeze(torch.stack(local_X).numpy()), local_y.numpy())

Alpha:[0.83882004 0.04029498 0.04029498 0.04029498 0.04029498]
Training Loss: 0.32651797
Alpha:[0.839726  0.0400685 0.0400685 0.0400685 0.0400685]
Training Loss: 0.27955872
Alpha:[0.83984923 0.04003768 0.04003768 0.04003768 0.04003768]
Training Loss: 0.26284653
Alpha:[0.8398874  0.04002814 0.04002814 0.04002814 0.04002814]
Training Loss: 0.2542592


In [10]:
predictions = onn_network.predict(X_test)
print("Accuracy: {}".format(accuracy_score(y_test, predictions)))

Accuracy: 0.9712


#Using contextual bandit - ONN_THS

In this example the ONN acts like a contextual bandits a reinforcement learning algorithm type. 

In [0]:
X_linear, Y_linear = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)
X_non_linear, Y_non_linear = make_circles(n_samples=10000, noise=0.1, factor=0.3, shuffle=True)
X_linear_2, Y_linear_2 = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)

X_linear_train = X_linear[:5000]
Y_linear_train = Y_linear[:5000]

X_linear_test = X_linear[5000:]
Y_linear_test = Y_linear[5000:]

X_non_linear_train = X_non_linear[:5000]
Y_non_linear_train = Y_non_linear[:5000]

X_non_linear_test = X_non_linear[5000:]
Y_non_linear_test = Y_non_linear[5000:]

X_linear_train_2 = X_linear_2[:5000]
Y_linear_train_2 = Y_linear_2[:5000]

X_linear_test_2 = X_linear_2[5000:]
Y_linear_test_2 = Y_linear_2[5000:]

In [0]:
gp = ONN_THS(2, 5, 100, 2)

In [5]:
for epoch in range(5):

    for i in range(len(X_linear_train)):
        x = np.asarray([X_linear_train[i, :]])
        y = np.asarray([Y_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_linear_test[i, :]]))[0])
          print("Accuracy: " + str(accuracy_score(Y_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.03783625
Accuracy: 0.938
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.1704272  0.20562138 0.20954046 0.20842516 0.20598587]
Training Loss: 2.8511312
Accuracy: 0.842
Alpha:[0.04529483 0.2372146  0.23887365 0.23777498 0.24084195]
Training Loss: 2.9067016
Alpha:[0.04934153 0.23565038 0.23894446 0.23790586 0.23815775]
Training Loss: 0.89143544
Accuracy: 0.7946
Alpha:[0.18905842 0.2011414  0.2039329  0.20293479 0.20293246]
Training Loss: 0.638691
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 1.2998967
Accuracy: 0.9372
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Accuracy: 0.5044
Alpha:[0.20965716 0.15684147 0.3117969  0.15653124 0.1651732 ]
Training Loss: 2.4024343
Alpha:[0.8399999 0.04      0.04      0.04      0.04     ]
Training Loss: 0.0
Ac

In [6]:
for epoch in range(5):

    for i in range(len(X_non_linear_train)):
        x = np.asarray([X_non_linear_train[i, :]])
        y = np.asarray([Y_non_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_non_linear_test[i, :]]))[0])
          print("Accuracy: " + str(accuracy_score(Y_non_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.8394417  0.0401068  0.040231   0.04010678 0.04011366]
Training Loss: 0.057519287
Accuracy: 0.4964
Alpha:[0.83992    0.04001539 0.04003379 0.04001539 0.04001539]
Training Loss: 0.18476209
Accuracy: 0.4928
Alpha:[0.8389778  0.04001202 0.04098608 0.04001202 0.04001202]
Training Loss: 0.14300129
Alpha:[0.8385863  0.04002043 0.04079157 0.04002043 0.04058118]
Training Loss: 0.14015199
Accuracy: 0.4926
Alpha:[0.8388008  0.0400123  0.04064631 0.0400123  0.04052823]
Training Loss: 0.1454754
Accuracy: 0.49
Alpha:[0.83798873 0.04046615 0.04053768 0.04046615 0.04054123]
Training Loss: 0.15644525
Accuracy: 0.502
Alpha:[0.8398817  0.04001858 0.04005715 0.04001858 0.04002398]
Training Loss: 0.16174349
Accuracy: 0.8878
Alpha:[0.83951735 0.04012064 0.04012064 0.04012064 0.04012064]
Training Loss: 0.19150087
Alpha:[0.8395833  0.04010417 0.04010417 0.04010417 0.04010417]
Training Loss: 0.2572823
Alpha:[0.8396205  0.04009486 0.04009486 0.04009486 0.04009486]
Training Loss: 0.21250293
Accuracy: 0.